In [1]:
#!pip install pandas rpy2 matplotlib

In [1]:
import rpy2.robjects as robjects
import numpy as np
import pandas as pd
from sklearn.linear_model import ElasticNet, SGDClassifier

In [2]:
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
pandas2ri.activate()

ro.r['load']("faces.RData")

'd'


In [3]:
df = ro.r['d']

In [4]:
type(df)

pandas.core.frame.DataFrame

In [5]:
ro.r['load']['vgg.RData']

TypeError: 'SignatureTranslatedFunction' object is not subscriptable

In [7]:
df.shape

(1085795, 42)

In [9]:
smaller_df = df.iloc[:10859]

In [11]:
smaller_df.to_csv('faces_smaller_chunk.csv')

In [40]:
faces = pd.read_csv('faces_smaller_chunk.csv')

In [41]:
faces.head()

,Unnamed: 0,userid,gender,age,country,facial_hair,pol,pol_dat_us,pol_dat_ca,pol_dat_uk,...,left_eye_status.no_glass_eye_open,left_eye_status.normal_glass_eye_close,left_eye_status.dark_glasses,right_eye_status.normal_glass_eye_open,right_eye_status.no_glass_eye_close,right_eye_status.occlusion,right_eye_status.no_glass_eye_open,right_eye_status.normal_glass_eye_close,right_eye_status.dark_glasses,ethnicity.value
0,1,1,0.0,NaN,united states,0.001713,liberal,NaN,NaN,NaN,...,1.8,0.0,0.0,0.0,0.0,99.9,0.1,0.0,0.0,black
1,2,2,1.0,NaN,united states,0.005151,liberal,NaN,NaN,NaN,...,100.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,black
2,3,3,0.0,NaN,united states,0.977098,conservative,NaN,NaN,NaN,...,0.3,0.0,15.5,39.7,0.0,0.0,0.0,0.0,60.2,india
3,4,4,1.0,24.36,united states,0.000478,liberal,NaN,NaN,NaN,...,98.9,0.0,0.5,0.0,0.0,10.2,81.7,0.0,8.1,black
4,5,5,1.0,NaN,united states,0.005617,liberal,NaN,NaN,NaN,...,0.0,0.0,0.0,2.3,0.0,97.3,0.1,0.1,0.2,asian


In [42]:
ground_truth = ["pol_dat_us", "pol_dat_ca","pol_dat_uk","pol_fb_us"]

In [43]:
cv_folds = 4

In [44]:
folds = np.random.randint(1,cv_folds+1, len(faces))

VGG is a concatenation of facial features generated from Face++

In [45]:
vgg = pd.read_csv('./vgg_chunks/chunk_1.csv', index_col=0, header=0);

In [46]:
vgg.shape

(10859, 2048)

In [47]:
i = ground_truth[3]
j = 1

In [48]:
col = f'pred_{i}'
col

'pred_pol_fb_us'

In [49]:
faces[col] = np.nan

In [50]:
faces[col]

0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
         ..
10854   NaN
10855   NaN
10856   NaN
10857   NaN
10858   NaN
Name: pred_pol_fb_us, Length: 10859, dtype: float64

In [51]:
not_na = faces[faces[i].notna()].index
not_na

Int64Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,
                9,
            ...
            10843, 10844, 10845, 10846, 10847, 10848, 10849, 10850, 10851,
            10852],
           dtype='int64', length=9429)

In [52]:
test_index = not_na[folds[not_na] == j]

train_index = not_na[folds[not_na] != j]

In [53]:
vgg.iloc[train_index]

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V2039,V2040,V2041,V2042,V2043,V2044,V2045,V2046,V2047,V2048
1,-0.464691,0.769671,0.285339,-0.690542,4.078915,-0.670584,-0.527952,-0.542535,-0.138685,0.190611,...,-0.351987,-0.239606,-0.294364,-0.467418,-0.316501,3.901776,-0.523199,0.877471,2.118653,-0.507040
2,-0.216150,-0.718548,-0.632259,-0.709603,0.418577,-0.708190,-0.526750,-0.542845,3.409459,-0.530470,...,-0.651550,-0.494568,-0.307999,-0.499397,1.684759,-0.478791,1.243010,1.172261,0.891900,-0.516734
3,1.794234,-0.722560,-0.611274,0.246038,1.952007,-0.687019,1.919408,-0.536251,0.228596,0.580031,...,-0.649727,-0.472004,-0.308215,0.516337,-0.570775,1.686378,-0.368707,0.705388,1.560745,-0.235392
4,-0.614232,-0.723645,-0.625850,-0.700769,0.890061,-0.708099,-0.517712,0.295749,-0.601133,-0.700777,...,-0.615763,-0.346717,-0.295181,1.710085,-0.606019,6.268199,-0.540629,0.191284,-0.658648,-0.508116
5,-0.340294,2.308569,-0.227442,0.966067,-0.129732,-0.697859,-0.226779,-0.548416,-0.312894,-0.400656,...,-0.645283,-0.061791,-0.307888,-0.174181,-0.576878,-0.374934,-0.537209,-0.327834,-0.034123,-0.432091
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10849,-0.615484,-0.636381,-0.603358,0.167440,-0.128089,0.777650,1.705667,-0.540217,-0.022750,-0.647462,...,-0.640117,-0.074280,-0.307193,-0.456712,-0.605985,-0.462110,0.586493,-0.140242,-0.543110,-0.518298
10850,0.019059,0.029472,-0.164447,0.101568,-0.339045,-0.481679,-0.415570,-0.504337,-0.529456,-0.516810,...,0.731238,0.173139,-0.308464,-0.469266,-0.604413,-0.352750,-0.541665,-0.573212,-0.523112,-0.517023
10851,-0.060267,-0.705025,-0.602377,1.079070,-0.534121,0.429713,-0.484970,-0.075476,-0.600968,-0.458346,...,-0.318451,3.351063,-0.252847,-0.432449,-0.605439,0.105908,-0.541663,-0.566663,-0.491385,-0.509674
10852,0.120538,-0.715838,1.009706,-0.712069,-0.397888,0.062870,-0.524864,-0.241348,-0.563695,-0.589039,...,0.468343,-0.392630,-0.308413,-0.499445,-0.246427,-0.474798,1.950026,-0.548894,-0.654915,-0.468898


In [54]:
x=vgg.iloc[train_index]
x.shape

(7100, 2048)

In [55]:
y=faces[i][train_index]
y.head()

0    0.0
1    0.0
2    1.0
3    0.0
4    0.0
Name: pol_fb_us, dtype: float64

In [56]:
x.shape, y.shape

((7100, 2048), (7100,))

In [57]:
(x.values).shape

(7100, 2048)

In [58]:
m = SGDClassifier(penalty='elasticnet')

In [59]:
m.fit(x, y)

SGDClassifier(penalty='elasticnet')

In [60]:
vgg.iloc[test_index].values.shape

(2329, 2048)

In [61]:
faces[col][test_index] = m.predict(vgg.iloc[test_index])

<ipython-input-61-71efe33e940c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  faces[col][test_index] = m.predict(vgg.iloc[test_index])


In [27]:
faces[faces[col].notna()]

,userid,gender,age,country,facial_hair,pol,pol_dat_us,pol_dat_ca,pol_dat_uk,pol_fb_us,...,left_eye_status.normal_glass_eye_close,left_eye_status.dark_glasses,right_eye_status.normal_glass_eye_open,right_eye_status.no_glass_eye_close,right_eye_status.occlusion,right_eye_status.no_glass_eye_open,right_eye_status.normal_glass_eye_close,right_eye_status.dark_glasses,ethnicity.value,pred_pol_fb_us
3,3,0.0,NaN,united states,0.977098,conservative,NaN,NaN,NaN,1.0,...,0.0,15.5,39.7,0.0,0.0,0.0,0.0,60.2,india,0.0
4,4,1.0,24.36,united states,0.000478,liberal,NaN,NaN,NaN,0.0,...,0.0,0.5,0.0,0.0,10.2,81.7,0.0,8.1,black,0.0
7,7,1.0,27.79,united states,0.055370,liberal,NaN,NaN,NaN,0.0,...,0.0,6.2,0.0,0.0,0.0,100.0,0.0,0.0,white,0.0
9,9,1.0,38.13,united states,0.003721,conservative,NaN,NaN,NaN,1.0,...,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,white,1.0
12,12,0.0,30.48,united states,0.103169,liberal,NaN,NaN,NaN,0.0,...,0.0,0.1,0.0,47.8,0.3,28.6,0.1,23.2,black,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10836,10836,1.0,25.14,united states,0.000036,liberal,NaN,NaN,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,white,0.0
10837,10837,0.0,NaN,united states,0.884177,conservative,NaN,NaN,NaN,1.0,...,0.0,0.0,0.1,93.9,0.7,0.8,1.3,3.2,india,0.0
10838,10838,1.0,NaN,united states,0.002126,conservative,NaN,NaN,NaN,1.0,...,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,white,0.0
10842,10842,1.0,25.62,united states,0.135778,liberal,NaN,NaN,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,white,0.0


Int64Index([    0,     6,    13,    17,    19,    23,    33,    34,    44,
               47,
            ...
            10842, 10847, 10851, 10852, 10853, 10854, 10855, 10856, 10857,
            10858],
           dtype='int64', length=3468)

In [77]:
for i in ground_truth:
    col="pred_"+i
    faces[[col]]=np.nan
    # print(faces[col])
    not_na = faces[faces[i].notna()].index

    for j in range(cv_folds):
        #Indices in R start at 1 so we must subtract 1 to keep it in bounds
        test_index = not_na[folds[not_na] == j]

        train_index = not_na[folds[not_na] != j]
        
        x=vgg.iloc[train_index]
        y=faces[i][train_index]
        
        test_x = vgg.iloc[test_index]
        
        if test_x.shape[0] > 0:
            m.fit(x, y)
            faces[col][test_index] = m.predict(test_x)
            
            is_na = faces[faces[col].isna()].index
            fx = is_na[folds[is_na] == 1]
            other_sample_x = vgg.iloc[fx]
            
            if other_sample_x.shape[0] > 0:
                faces[col][fx] = m.predict(other_sample_x)

/home/21rfk/.local/lib/python3.8/site-packages/sklearn/base.py:441: UserWarning: X does not have valid feature names, but SGDClassifier was fitted with feature names
  warnings.warn(
<ipython-input-77-27de5a1af4cf>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  faces[col][test_index] = m.predict(test_x)
<ipython-input-77-27de5a1af4cf>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  faces[col][fx] = m.predict(other_sample_x)
/home/21rfk/.local/lib/python3.8/site-packages/sklearn/base.py:441: UserWarning: X does not have valid feature names, but SGDClassifier was fitted with feature names
  warnings.warn(
<ipython-i

In [78]:
faces

,Unnamed: 0,userid,gender,age,country,facial_hair,pol,pol_dat_us,pol_dat_ca,pol_dat_uk,...,right_eye_status.no_glass_eye_close,right_eye_status.occlusion,right_eye_status.no_glass_eye_open,right_eye_status.normal_glass_eye_close,right_eye_status.dark_glasses,ethnicity.value,pred_pol_fb_us,pred_pol_dat_us,pred_pol_dat_ca,pred_pol_dat_uk
0,1,1,0.0,NaN,united states,0.001713,liberal,NaN,NaN,NaN,...,0.0,99.9,0.1,0.0,0.0,black,NaN,NaN,NaN,NaN
1,2,2,1.0,NaN,united states,0.005151,liberal,NaN,NaN,NaN,...,0.0,0.0,100.0,0.0,0.0,black,0.0,NaN,NaN,NaN
2,3,3,0.0,NaN,united states,0.977098,conservative,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,60.2,india,0.0,NaN,NaN,NaN
3,4,4,1.0,24.36,united states,0.000478,liberal,NaN,NaN,NaN,...,0.0,10.2,81.7,0.0,8.1,black,0.0,NaN,NaN,NaN
4,5,5,1.0,NaN,united states,0.005617,liberal,NaN,NaN,NaN,...,0.0,97.3,0.1,0.1,0.2,asian,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10854,10855,10855,1.0,NaN,united states,0.402065,liberal,0.0,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,white,NaN,1.0,NaN,NaN
10855,10856,10856,1.0,40.00,united states,0.076524,liberal,0.0,NaN,NaN,...,0.5,56.0,42.0,0.2,1.2,black,NaN,NaN,NaN,NaN
10856,10857,10857,0.0,NaN,united kingdom,0.751074,liberal,NaN,NaN,0.0,...,0.0,0.0,100.0,0.0,0.0,white,NaN,NaN,NaN,0.0
10857,10858,10858,1.0,35.00,united states,0.010946,conservative,1.0,NaN,NaN,...,0.1,0.4,97.1,0.0,2.4,white,NaN,0.0,NaN,NaN
